## Checkpoint 2

In [79]:
from sklearn 
import cross_validation, ensemble, grid_search, linear_model, metrics, pipeline, preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
import xgboost as xgb
%matplotlib inline

In [2]:
data = pd.read_csv('lar_flat.csv', header=0)

In [3]:
categorical =[
     'loan_type',
     'loan_purpose',
     'preapproval',
     'action_type',
     'county_code',
     'applicant_ethnicity',
     'applicant_race_1',
     'applicant_sex',
     'lien_status']
categorical
categorical.remove( 'action_type')
categorical

['loan_type',
 'loan_purpose',
 'preapproval',
 'county_code',
 'applicant_ethnicity',
 'applicant_race_1',
 'applicant_sex',
 'lien_status']

## 5., 8. remove: income ,Tract_To_MSAMD_Income_pct, Minority_Population_pct = NA

In [4]:
data = data[data.applicant_income_ink != 'NA  ']
data = data[data.tract_to_msamd_income_pct != 'NA    ']
data = data[data.minority_population_pct != 'NA    ']
data=data[data.lien_status != '']

numerical = [x for x in data.columns.values if x not in categorical][1:]
numerical.remove('rate_spread')
numerical
numerical.remove('action_type')


## rate_spread only 219 non-ambigous records, so will be removed for now

In [5]:
test = data[data.rate_spread != 'NA   '].rate_spread
test.size
data = data.drop(['rate_spread'], axis=1)
data = data.drop(['Unnamed: 0'], axis=1)

## now we have our data preprocessed

# Categirical data values
#### action_type: 'Withdrawn', 'Denied', 'Approved Not Accepted', 'Originated'
#### loan_type: 'Conventional', 'FHA', 'VA', 'FSA.RHS'
#### loan_purpose: 'Refinancing', 'Home improvement', 'Home purchase'
#### preapproval: 'Not applicable', 'Not requested', 'Requested'
#### country_code: '20', '170', '020', '090', '110', '122', '220', '150', '100', '130','261', '280', '240','201', '164', '230', '188', '180', '068','013', '290', '050', '185', '070', '060', '105', '016', '282', '270'
#### applicant_ethnicity: 'Non Hispanic.Latino', 'No Info', 'Hispanic.Latino'
#### plicant_race_1: 'White', 'Asian', 'No Info', 'AmerInd.AlaskaNat','Hawaiian.PacificIs', 'Black.AfroAmer'
#### applicant_sex: 'Male', 'Female', 'No Info'
#### lien_status: 'First lien', 'Subordinate lien', 'No lien'

# Model

In [6]:
#10% for the test
# factor = np.random.rand(len(data)) < 0.9
# train = data[factor]
# test = data[~factor]
# train_labels = train.action_type.values
# test_labels = test.action_type.values

# one_hot = pd.get_dummies(train_labels)
# type(one_hot)

# train_data = train.drop(['action_type'],axis = 1)
# test_data = test.drop(['action_type'],axis = 1)

In [7]:
data_categorical = data[categorical]
data_numerical = data[numerical]
labels = data[['action_type']]


In [8]:
for num in numerical:
    scaler = preprocessing.StandardScaler(with_mean=0.0,with_std=1.0)
    scaler.fit(data_numerical[[num]])
    data_numerical[[num]] = scaler.transform(data_numerical[[num]])


/home/uri/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/uri/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
# encoder = preprocessing.LabelEncoder()
# encoder.fit(train_data_categorical)
# X_train_categorical = encoder.transform(train_data_categorical)
# X_test_categorical = encoder.transform(test_data_categorical)

for category in categorical:
    encoder = preprocessing.LabelEncoder()
    encoder.fit(data_categorical[[category]])
    data_categorical[[category]] = encoder.transform(data_categorical[[category]])


/home/uri/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/uri/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/uri/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:2382: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.ix._setitem_with_indexer((slice(None), indexer), value)
/home/uri/anaconda2/lib/python2

In [10]:
data_categorical.head()

,loan_type,loan_purpose,preapproval,county_code,applicant_ethnicity,applicant_race_1,applicant_sex,lien_status
0,0,2,0,19,2,5,1,0
1,0,2,0,19,2,5,1,0
2,0,2,0,15,2,5,1,0
3,0,2,0,19,2,5,1,0
4,0,2,0,15,2,5,1,0


In [11]:
# encoder = preprocessing.LabelEncoder()
# encoder.fit_transform(test_labels_categorical)
# encoder.fit_transform(test_labels_numerical)
encoder = preprocessing.LabelEncoder()
labels = encoder.fit_transform(labels)

/home/uri/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:125: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
data[numerical] = data_numerical[numerical]
data[categorical] = data_categorical[categorical]
data = data.drop('action_type',axis=1)
data.head()

,loan_type,loan_purpose,loan_amount_ink,preapproval,county_code,applicant_ethnicity,co_applicant_ethnicity,applicant_race_1,applicant_sex,applicant_income_ink,hoepa_status,lien_status,minority_population_pct,hud_median_family_income,tract_to_msamd_income_pct,number_of_owner_occupied_units
0,0,2,1.610946,0,19,2,3.399021,5,1,1.005958,323.833291,0,2.632645,17.15497,3.619933,1.061290
1,0,2,1.061964,0,19,2,1.359608,5,1,1.297686,323.833291,0,1.104669,17.15497,4.486749,2.439991
2,0,2,1.601946,0,15,2,1.359608,5,1,1.327865,323.833291,0,1.319718,17.15497,3.614508,2.547423
3,0,2,0.683977,0,19,2,1.359608,5,1,1.317805,323.833291,0,1.111460,17.15497,5.257854,2.820884
4,0,2,1.196960,0,15,2,1.359608,5,1,0.754469,323.833291,0,0.917540,17.15497,4.131033,3.926124


In [13]:
print type(labels)

<type 'numpy.ndarray'>


In [14]:
train = data.iloc[:-2000]
test = data.iloc[-2000:]

train_labels = labels[:-2000]
test_labels = labels[-2000:]

In [84]:
model = ensemble.RandomForestClassifier(random_state=777,
                                        max_depth=8,
                                        criterion='gini',
                                        n_estimators=183,
                                        n_jobs=-1)
model.fit(train,train_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=183, n_jobs=-1,
            oob_score=False, random_state=777, verbose=0, warm_start=False)

## accuracy

In [85]:
print 'accuracy = ' ,metrics.accuracy_score(test_labels, model.predict(test))
print 'precision = ', metrics.precision_score(test_labels, model.predict(test),average='weighted',labels=list(set(labels)))

accuracy =  0.7295
precision =  0.676493098813


## grid search

In [86]:
model.get_params().keys()

['warm_start',
 'oob_score',
 'n_jobs',
 'verbose',
 'max_leaf_nodes',
 'bootstrap',
 'min_samples_leaf',
 'n_estimators',
 'min_samples_split',
 'min_weight_fraction_leaf',
 'criterion',
 'random_state',
 'max_features',
 'max_depth',
 'class_weight']

In [87]:
grid_params = {
   # 'model_fitting__loss' : ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'n_estimators' : np.linspace(50, 200, num = 10,dtype='int'),
    'max_depth': np.linspace(3,11,dtype='int')
}

In [88]:
randomized_grid = grid_search.RandomizedSearchCV(model, grid_params, scoring = 'accuracy',n_iter = 20)
randomized_grid.fit(train, train_labels)
print randomized_grid.best_score_
print randomized_grid.best_params_

0.549554013875
{'n_estimators': 50, 'max_depth': 3}


In [89]:
model = xgb.XGBClassifier(learning_rate =0.1, 
                         n_estimators=140, 
                         max_depth=5,
                         min_child_weight=1, 
                         gamma=0,
                         subsample=0.8, 
                         colsample_bytree=0.8,
                         nthread=4, 
                         scale_pos_weight=1, 
                         seed=27)

In [90]:
model.fit(train, train_labels)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=140, nthread=4,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8)

In [92]:
print 'accuracy = ' ,metrics.accuracy_score(test_labels, model.predict(test))
print 'precision = ', metrics.precision_score(test_labels, model.predict(test),average='weighted',labels=list(set(labels)))

accuracy =  0.4065
precision =  0.535355908803


In [93]:
model.get_params().keys()

['reg_alpha',
 'colsample_bytree',
 'silent',
 'colsample_bylevel',
 'scale_pos_weight',
 'learning_rate',
 'missing',
 'max_delta_step',
 'nthread',
 'base_score',
 'n_estimators',
 'subsample',
 'reg_lambda',
 'seed',
 'min_child_weight',
 'objective',
 'max_depth',
 'gamma']

In [114]:
grid_params = {
   'subsample': [0.4,0.5,0.6,0.7,0.8],
   'max_depth': np.linspace(2, 10),
   'learning_rate': np.linspace(0.001, 0.07),
    'n_estimators' : np.linspace(100,200,10,dtype=int)
}

In [ ]:
gsearch = grid_search.GridSearchCV(model,grid_params,n_jobs=-1,cv=2)
gsearch.fit(train, train_labels)


In [113]:
print gsearch.best_score_
print gsearch.best_params_

0.650561612157
{'subsample': 0.8, 'learning_rate': 0.05, 'max_depth': 1.0}
